# Summarize K-Means (K=10) Cluster for full-bow

In [1]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict
import json


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm

import scipy.sparse
import random
import itertools
from itertools import combinations


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *
from helpful_functions import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"
csv_dir_mod = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_modularity/"
csv_dir_walk = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_walktrap/"
csv_dir_full_tfidf_km10 = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_full_tfidf_km10/"
csv_dir_full_tfidf_km100 = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_full_tfidf_km100/"
csv_dir_full_bow_km10 = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_full_bow_km10/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# clusters directory
clusters_dir = "C:/Users/Michael/Desktop/network_data/raw/scotus/clusters/"

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

## load bow vectors
**bow_matrix** = (row_index, column_index): token/word-count value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in bow_matrix (correspond to column indices)

In [3]:
def load_bow(nlp_dir):
    """
    bow_matrix, op_id_to_bow_id = load_bow(nlp_dir)
    """
    bow_matrix = load_sparse_csr(nlp_dir + 'bag_of_words_matrix.npz')

    with open(nlp_dir + 'op_id_to_bow_id.p', 'rb') as f:
        op_id_to_bow_id = pickle.load(f)

    with open(nlp_dir + 'vocab.p', 'rb') as f:
        vocab = pickle.load(f)

    return bow_matrix, op_id_to_bow_id, vocab

In [4]:
bow_matrix, op_id_to_bow_id, vocab = load_bow(nlp_bow_dir)

In [5]:
bow_matrix

<27885x567570 sparse matrix of type '<type 'numpy.int64'>'
	with 20817470 stored elements in Compressed Sparse Row format>

In [6]:
clusters = pd.read_csv(csv_dir + 'clusters_full_bow.csv')
clusters.head()

,Unnamed: 0,mod,wt,km_10,km_100
0,145658,1,5,0,32
1,89370,3,294,0,32
2,89371,0,35,0,32
3,89372,0,3,0,32
4,89373,0,3,0,32


In [7]:
nlp_tfidf_clusters = clusters['km_10'].tolist()

nlp_clusters = pd.Series(nlp_tfidf_clusters, index=op_id_to_bow_id)
nlp_clusters.head()

145658    0
89370     0
89371     0
89372     0
89373     0
dtype: int64

## get the top 5 biggest clusters

In [8]:
'''
dict_top_n_clusters = dictionary of top K clusters 
                      (key=cluster #, value=opinions in cluster)
                      
biggest_n_clusters = list of top K clusters (int)
'''

dict_top_n_clusters, biggest_n_clusters = get_top_n_clusters(5, 10, nlp_clusters)

cluster 0 : 14270 opinions
cluster 8 : 7428 opinions
cluster 4 : 2940 opinions
cluster 7 : 1232 opinions
cluster 3 : 1111 opinions


# Top K Words of Each Cluster
This function summarizes a set of opinions by returning the words that appear in these opinions with the highest tf-idf scores.

# Top K Words ($\mu_{cluster}$) of Each Cluster
compute the mean tf-idf vector of the cluster, return the top K words from this mean vector

# Top K Words ($\mu_{cluster} - \mu_{complement}$ ) of Each Cluster
compute the mean tf-idf vector of the cluster and also of the complement of the cluster,  
take the difference mu_cluster - mu_complement, return the top K words in this difference

# Most Relevant Opinion of Each Cluster
compute the mean tf-idf vector, return the document in the cluster closet to the mean  

# Cluster 0 Summary (14270 opinions)

In [9]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[0]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_bow_km10 + "cluster_0.csv")

print cluster_info.shape

(14270, 3)
Wall time: 7.31 s


In [10]:
cluster_info = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_0.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,DaimlerChrysler Corp. v. Cuno,2006-05-15,https://www.courtlistener.com/opinion/145658/d...
1,1,Garfielde v. United States,1876-05-18,https://www.courtlistener.com/opinion/89370/ga...
2,2,Whiteside v. United States,1876-11-27,https://www.courtlistener.com/opinion/89371/wh...
3,3,Barkley v. Levee Commissioners,1876-12-18,https://www.courtlistener.com/opinion/89372/ba...
4,4,Broughton v. Pensacola,1876-12-18,https://www.courtlistener.com/opinion/89373/br...


In [11]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[0], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[0], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[0], all_the_opinions, k, bow_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[0], bow_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_bow_km10 + "cluster_0_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 3min 17s


In [12]:
cluster_summary = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_0_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,state,court,court,104977
1,1,feet,state,contract,104977
2,2,law,v,prison,104977
3,3,punish,case,habea,104977
4,4,contract,act,error,104977


# Cluster 8 Summary (7248 opinions)

In [13]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[8]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_bow_km10 + "cluster_8.csv")

print cluster_info.shape

(7428, 3)
Wall time: 3.65 s


In [14]:
cluster_info = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_8.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Dalton v. Jennings,1876-12-18,https://www.courtlistener.com/opinion/89374/da...
1,1,"The"" Atlas""",1876-11-27,https://www.courtlistener.com/opinion/89379/th...
2,2,Wilson v. Daniel,1798-08-17,https://www.courtlistener.com/opinion/2224796/...
3,3,Griffin v. McCoach,1941-06-02,https://www.courtlistener.com/opinion/103543/g...
4,4,Rogers v. Ritter,1871-11-20,https://www.courtlistener.com/opinion/88395/ro...


In [15]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[8], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[8], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[8], all_the_opinions, k, bow_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[8], bow_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_bow_km10 + "cluster_8_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 57 s


In [16]:
cluster_summary = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_8_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,school,court,tax,102039
1,1,grant,state,act,102039
2,2,right,v,said,102039
3,3,district,case,public,102039
4,4,candid,act,bond,102039


# Cluster 4 Summary (2940 opinions)

In [17]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[4]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_bow_km10 + "cluster_4.csv")

print cluster_info.shape

(2940, 3)
Wall time: 1.44 s


In [18]:
cluster_info = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_4.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Alabama v. King & Boozer,1941-11-10,https://www.courtlistener.com/opinion/103545/a...
1,1,Fowler v. Merrill,1851-03-18,https://www.courtlistener.com/opinion/86651/fo...
2,2,Gaines v. Fuentes,1876-03-20,https://www.courtlistener.com/opinion/89236/ga...
3,3,Moore v. Mississippi,1875-04-19,https://www.courtlistener.com/opinion/89041/mo...
4,4,Insurance Co. v. Francis,1871-03-18,https://www.courtlistener.com/opinion/88319/in...


In [19]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[4], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[4], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[4], all_the_opinions, k, bow_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[4], bow_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_bow_km10 + "cluster_4_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 24.3 s


In [20]:
cluster_summary = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_4_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,state,court,state,104441
1,1,mine,state,sct,104441
2,2,district,v,us,104441
3,3,power,case,v,104441
4,4,school,us,district,104441


# Cluster 7 Summary (1232 opinions)

In [21]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[7]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_bow_km10 + "cluster_7.csv")

print cluster_info.shape

(1232, 3)
Wall time: 550 ms


In [22]:
cluster_info = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_7.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Bernards v. Johnson,1941-11-10,https://www.courtlistener.com/opinion/103547/b...
1,1,Reetz v. Michigan,1903-02-23,https://www.courtlistener.com/opinion/95797/re...
2,2,Black v. Amen,1958-03-03,https://www.courtlistener.com/opinion/105639/b...
3,3,Cain v. Kentucky,1970-03-30,https://www.courtlistener.com/opinion/108104/c...
4,4,Smith v. United States,1896-03-02,https://www.courtlistener.com/opinion/94364/sm...


In [23]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[7], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[7], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[7], all_the_opinions, k, bow_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[7], bow_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_bow_km10 + "cluster_7_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 13.3 s


In [24]:
cluster_summary = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_7_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,state,state,state,87419
1,1,power,court,law,87419
2,2,law,v,commerc,87419
3,3,commerc,case,power,87419
4,4,congress,act,act,87419


# Cluster 3 Summary (1111 opinions)

In [25]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[3]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_bow_km10 + "cluster_3.csv")

print cluster_info.shape

(1111, 3)
Wall time: 490 ms


In [26]:
cluster_info = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_3.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Central Stock Yards Co. v. Louisville & Nashvi...,1904-02-23,https://www.courtlistener.com/opinion/96013/ce...
1,1,Fountain v. Filson,1949-05-31,https://www.courtlistener.com/opinion/104659/f...
2,2,United States v. Mooney,1885-12-14,https://www.courtlistener.com/opinion/91505/un...
3,3,Barrett v. United States (No. 1),1898-02-21,https://www.courtlistener.com/opinion/94813/ba...
4,4,United States v. Eichman,1990-06-11,https://www.courtlistener.com/opinion/112453/u...


In [27]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[3], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[3], k, bow_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[3], all_the_opinions, k, bow_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[3], bow_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_bow_km10 + "cluster_3_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 12 s


In [28]:
cluster_summary = pd.read_csv(csv_dir_full_bow_km10 + 'cluster_3_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,state,court,land,95116
1,1,new,state,unit,95116
2,2,unit,v,car,95116
3,3,1,case,grant,95116
4,4,territori,act,sentenc,95116
